## Baseline Code

In [42]:
import random
import pandas as pd
import numpy as np
import os


In [22]:
# Fixed Random-Seed(랜덤 함수들이 동일한 시드에서 같은 랜덤값을 생성함.)
def seed_everything(seed):
    random.seed(seed) # random 모듈의 시드 설정
    os.environ['PYTHONHASHSEED'] = str(seed) # 환경변수 설정
    np.random.seed(seed) # Numpy에서 제공하는 랜덤 함수들이 동일한 시드에서 같은 랜덤 값을 생성함.

    seed_everything(42) # Seed 고정

## 데이터 불러오기

In [59]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

## 데이터 전처리

In [24]:
train_df = train_df.fillna(0)

In [25]:
train_df = train_df.rename(columns={ #train_df 열이름 변경
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})

train_df.drop(['num_date_time'], axis = 1, inplace =True)  # num_date_time(= building_number + date_time)이므로, 제거

test_df = test_df.rename(columns={ #test_df 열이름 변경
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity'
})

test_df.drop('num_date_time', axis = 1, inplace =True) # num_date_time(= building_number + date_time)이므로, 제거

In [31]:
# datetime을 년도, 월, 주, 시간으로 쪼갠다.
train_df['date_time'] = pd.to_datetime(train_df['date_time'], format = '%Y%m%d %H')
test_df['date_time'] = pd.to_datetime(test_df['date_time'], format = '%Y%m%d %H')



# date time feature 생성
train_df['hour'] = train_df['date_time'].dt.hour
train_df['day'] = train_df['date_time'].dt.day
train_df['month'] = train_df['date_time'].dt.month
train_df['year'] = train_df['date_time'].dt.year

test_df['hour'] = test_df['date_time'].dt.hour
test_df['day'] = test_df['date_time'].dt.day
test_df['month'] = test_df['date_time'].dt.month
test_df['year'] = test_df['date_time'].dt.year

## 모델링

In [45]:
train_X = train_df.drop(columns = ['date_time', 'sunshine', 'solar_radiation', 'power_consumption'])
train_y = train_df['power_consumption']

In [53]:
test = test_df.drop(columns=['date_time'])

## Regression Model Fit

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [48]:
model = RandomForestRegressor()
model.fit(train_X, train_y)

RandomForestRegressor()

In [54]:
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.3, random_state=42)

In [56]:
model.score(test_X, test_y)

0.9980241265896781

In [57]:
preds = model.predict(test)

In [62]:
submission['answer'] = preds

In [63]:
submission.to_csv('./data/basline_submission.csv', index = False)

In [71]:
train_X

,building_number,temperature,rainfall,windspeed,humidity,hour,day,month,year
77350,38,25.0,0.0,2.2,77.0,22,17,8,2022
126739,63,25.1,0.0,3.3,59.0,19,11,6,2022
17016,9,24.9,0.5,4.7,97.0,0,30,6,2022
130893,65,19.4,0.0,2.1,82.0,21,14,6,2022
111316,55,23.8,0.4,0.5,99.0,4,19,7,2022
...,...,...,...,...,...,...,...,...,...
84830,42,31.7,0.0,3.2,60.0,14,20,7,2022
67507,34,23.4,0.0,3.8,60.0,19,8,6,2022
143996,71,28.2,0.0,1.6,63.0,20,20,7,2022
180517,89,29.2,0.0,1.7,66.0,13,12,7,2022


0.9980241265896781
1.0
